# Pose Estimation with camera in real time

In [18]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")
cap = cv2.VideoCapture(1)

while True:
    ok, frame = cap.read()
    if not ok: break
    r = model(frame, verbose=False)[0]
    cv2.imshow("pose", r.plot())
    if cv2.waitKey(1) == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


# Showing only Keypoints

In [19]:
import cv2, numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

# optional: pyvirtualcam for virtual webcam output
try:
    import pyvirtualcam
    from pyvirtualcam import PixelFormat
    HAVE_PYVIRTUALCAM = True
except Exception:
    HAVE_PYVIRTUALCAM = False

model = YOLO("yolov8n-pose.pt")
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    # fallback to camera 0 if 1 is not available
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("No camera available")

# grab a first frame to determine size for the virtual camera
ok, frame = cap.read()
if not ok:
    cap.release()
    raise RuntimeError("Can't read from camera")

height, width = frame.shape[:2]

cam = None
if HAVE_PYVIRTUALCAM:
    try:
        # use RGB pixel format (pyvirtualcam expects RGB frames)
        cam = pyvirtualcam.Camera(width=width, height=height, fps=20, fmt=PixelFormat.RGB)
        print('pyvirtualcam opened:', cam.device)
    except Exception as e:
        print('Failed to open virtual camera:', e)
        cam = None

while True:
    ok, frame = cap.read()
    if not ok: break

    res = model(frame, verbose=False)[0]
    canvas = np.zeros_like(frame)                     # schwarzer Hintergrund
    ann = Annotator(canvas, line_width=2)

    # draw keypoints if present
    if hasattr(res, 'keypoints') and res.keypoints is not None:
        for kp in res.keypoints.xy:                       # kp: (17,2) für COCO
            ann.kpts(kp, radius=3)                        # zeichnet Punkte + Bones

    out = ann.result()

    # send to virtual camera (pyvirtualcam expects RGB order)
    if HAVE_PYVIRTUALCAM and cam is not None:
        try:
            rgb = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
            cam.send(rgb)
            cam.sleep_until_next_frame()
        except Exception as e:
            print('pyvirtualcam send failed:', e)
            HAVE_PYVIRTUALCAM = False

    # local preview
    cv2.imshow("pose", out)
    if cv2.waitKey(1) == 27: break  # ESC

cap.release()
if cam is not None:
    cam.close()
cv2.destroyAllWindows()


pyvirtualcam opened: OBS Virtual Camera
